In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import copy
# warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 80

### Эксперимент

#### Фиксированные параметры генератора
- series_type = "default"  // обычная линейная регрессия

- dim=10 // размерность сигнала
- low=-10, high=10 // границы весов для получения отклика
- noise_var=1 // дисперсия нормального шума
- workers_num=3 // число различных генераторов
- lower_bound=100, upper_bound=400 // границы длин каждого из кусков, получаемых генератором
- alternating=True // нет двух подряд кусков одного типа
#### Фиксированные параметры алгоритма
- from_start=False // начинаем подсчет всех потерь (и составного эксперта) не с самого начала, а после того как прошли по одному разу учатки каждого из  генераторов

- length=1500 // длина изучаемого участка временного ряда

- a=-40, b=40 // параметры алгоритма, отклики отсеиваем тоже по ним

#### Гиперпараметры эксперимента
- windows = [5, 10, 20] // варьируем длину окна обучения ($\frac12x, 1x, 2x$ соответственно, где $x$  &mdash; dim  сигнала )
- weights_function // функция начальных весов, cравниваем $\dfrac1{x^\alpha}$ для разных $\alpha$ и дефолтную $\dfrac{1}{(x+1)\ln^2(x+1)}$
- alpha_function // функция коэффицента $\alpha$ в Mixing Update, cравниваем $\dfrac1{(1 + x)^\alpha}$ для разных $\alpha$

In [34]:
df = pd.read_csv("../results/df-other.csv", index_col=0, sep='\t')
# df.loc[[5, 2, 13, 10, 21, 18]]
df

,train_window,weight_function,alpha_function,mean,random_1,random_2,random_3,random_4
5,5,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),235135.48,239444.02,198209.12,267607.19,235281.57
3,5,1 / (x^1.001),1 / (x + 1),190819.44,190700.95,162997.91,210267.49,199311.42
7,5,1 / (x^1.01),1 / (x + 1),190184.28,190122.27,162227.69,209655.72,198731.46
4,5,1 / (x^1.01),1 / (x + 1)^1.3,166154.00,170966.07,134417.11,185983.37,173249.44
6,5,1 / (x^1.01),1 / ((x + 1)*(ln(x + 1))^2),135296.21,165184.09,40253.48,174482.82,161264.46
2,5,1 / (x^1.01),1 / e^x,133224.39,165339.42,37392.26,171568.82,158597.05
0,5,1 / (x^1.01),1 / e^(x/3),133225.94,165340.95,37392.05,171568.85,158601.91
1,5,1 / (x^1.1),1 / (x + 1),194532.69,195382.82,164837.91,215724.05,202185.99
13,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),183257.14,180371.38,162705.30,199049.29,190902.58
11,10,1 / (x^1.001),1 / (x + 1),129067.96,120891.72,116613.22,134385.77,144381.15


In [4]:
df_w = df[df["alpha_function"] == "1 / (x + 1)"].iloc[:, :4]
df_w

,train_window,weight_function,alpha_function,mean
5,5,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),235135.48
3,5,1 / (x^1.001),1 / (x + 1),190819.44
7,5,1 / (x^1.01),1 / (x + 1),190184.28
1,5,1 / (x^1.1),1 / (x + 1),194532.69
13,10,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),183257.14
11,10,1 / (x^1.001),1 / (x + 1),129067.96
15,10,1 / (x^1.01),1 / (x + 1),128692.52
9,10,1 / (x^1.1),1 / (x + 1),134971.88
21,20,1 / ((x + 1) * (ln(x + 1))^2),1 / (x + 1),182775.02
19,20,1 / (x^1.001),1 / (x + 1),126549.10


In [28]:
df_a = df[(df["weight_function"] == "1 / (x^1.01)") & ((df["alpha_function"] == "1 / (x + 1)") | (df["alpha_function"] == "1 / e^x"))]
dfa = df_a[["alpha_function", "mean"]]


print(dfa.to_latex(index=False))

\begin{tabular}{lr}
\toprule
alpha_function & mean \\
\midrule
1 / (x + 1) & 190184.280000 \\
1 / e^x & 133224.390000 \\
1 / (x + 1) & 128692.520000 \\
1 / e^x & 142611.450000 \\
1 / (x + 1) & 126841.610000 \\
1 / e^x & 214711.720000 \\
\bottomrule
\end{tabular}



In [5]:
df[["train_window", "mean"]].groupby("train_window").mean()

,mean
train_window,
5,117750.785556
10,72380.342222
20,86215.462222


#### Выводы

##### Train window
В среднем, при окне обучения равном dim=10, результаты оказались лучше чем для размеров 5 и 20.

##### Weights function
При всех рассмотренных окнах, регрет для функции весов $\dfrac1{x^\alpha}$ меньше, чем для дефолтной $\dfrac{1}{(x+1)\ln^2(x+1)}$, и при стремлении $\alpha$ к 1 регрет уменьшается. То есть минимальный регрет среди рассмотренных здесь достигается  для $\dfrac1{x^{1.05}}$, она же была мной зафиксирована для экспериментами над alpha_function.

##### Alpha function
При окнах обучения $5$ и $10$, побеждают $\dfrac1{(x+1)^{1.5}}$ и $\dfrac1{(x+1)^{1.3}}$, но отрыв от остальных не слишком велик. \
При окне $20$ побеждает дефолтная $\dfrac1{(x+1)}$ c нарастающим отрывом при увеличении $\alpha$

**Таким образом, на практике лучшей функцией для начальных весов является $\dfrac1{x^{1+\varepsilon}}$ c как можно меньшим $\varepsilon$**

**Значительных улучшений дефолтной функции $\dfrac1{(x+1)}$ для коэффицента $\alpha$ Mixing Update  пока нет**

Теоретическая оценка сверху выглядит нормально: